<a href="https://colab.research.google.com/github/YounSooKimTech/NLP_Power/blob/main/04_18_Enron_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Status dataset

In [1]:
import pandas as pd

status = pd.read_csv('https://raw.githubusercontent.com/burgersmoke/enron-formality/master/enron_employee_positions/diesner_employee_email_positions.csv', names=["Name", "Email", "Position", "Rank"])

print(status.Position.value_counts())

status.head()

Vice President         40
Employee               39
Director               26
Manager                20
Trader                 12
Managing Director      11
CEO                    10
President               9
In House Lawyer         3
Director of Trading     1
Name: Position, dtype: int64


,Name,Email,Position,Rank
0,Andrea Ring,andrea.ring@enron.com,NaN,0
1,Andrew Lewis,andrew.lewis@enron.com,Vice President,4
2,Andy Zipper,andy.zipper@enron.com,Vice President,4
3,Barry Tycholiz,barry.tycholiz@enron.com,Vice President,4
4,Benjamin Rogers,benjamin.rogers@enron.com,NaN,0


In [2]:
status = status.dropna(subset=["Position"])
status.Position.value_counts()

Vice President         40
Employee               39
Director               26
Manager                20
Trader                 12
Managing Director      11
CEO                    10
President               9
In House Lawyer         3
Director of Trading     1
Name: Position, dtype: int64

# Enron email dataset

- Before 2001-05-01
- Both sender and receivers are from Enron corp
- n_receiver=1, n_sender=1, CC= None

In [3]:
import warnings
warnings.filterwarnings("ignore")


{"username":"kim4321","key":"3440b23b7ad1251f54c78b0330d0e03a"}

! pip install opendatasets
import opendatasets as od
import pandas
import re  

od.download('https://www.kaggle.com/datasets/wcukierski/enron-email-dataset')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: kim4321
Your Kaggle Key: ··········


100%|██████████| 358M/358M [00:04<00:00, 77.5MB/s]


In [4]:
import pandas as pd

df = pd.read_csv('/content/enron-email-dataset/emails.csv')

print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517401 entries, 0 to 517400
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   file     517401 non-null  object
 1   message  517401 non-null  object
dtypes: object(2)
memory usage: 7.9+ MB
None


,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [5]:
# use the library to parse the dataset
import email
import pandas as pd
from email import utils

def extract_email_info(email_string):
    email_message = email.message_from_string(email_string)
    sender = email_message['From']
    receiver = email_message["To"]
    n_receivers = len(utils.getaddresses([email_message['To']]))
    CC = email_message["CC"]
    subject = email_message['Subject']
    date = utils.parsedate_to_datetime(email_message['Date'])
    # Extract email body
    body = ""
    if email_message.is_multipart():
        for part in email_message.get_payload():
            if part.get_content_type() == 'text/plain':
                body = part.get_payload()
                break
    else:
        body = email_message.get_payload()
    return sender, receiver, n_receivers, CC, subject, body, date

df[['Sender',"Receiver", 'n_receivers', 'CC', 'Subject', 'Body', "Date"]] = df['message'].apply(extract_email_info).apply(pd.Series)

df.head()

,file,message,Sender,Receiver,n_receivers,CC,Subject,Body,Date
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,phillip.allen@enron.com,tim.belden@enron.com,1,None,,Here is our forecast\n\n,2001-05-14 16:39:00-07:00
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,phillip.allen@enron.com,john.lavorato@enron.com,1,None,Re:,Traveling to have a business meeting takes the...,2001-05-04 13:51:00-07:00
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,phillip.allen@enron.com,leah.arsdall@enron.com,1,None,Re: test,test successful. way to go!!!,2000-10-18 03:00:00-07:00
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,phillip.allen@enron.com,randall.gay@enron.com,1,None,,"Randy,\n\n Can you send me a schedule of the s...",2000-10-23 06:13:00-07:00
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,phillip.allen@enron.com,greg.piper@enron.com,1,None,Re: Hello,Let's shoot for Tuesday at 11:45.,2000-08-31 05:07:00-07:00


In [6]:
# check the body

print(df.loc[1750, "Body"])

---------------------- Forwarded by Phillip K Allen/HOU/ECT on 03/13/2000 
05:33 PM ---------------------------
   
	
	
	From:  William Kelly                           03/13/2000 01:43 PM
	

To: Phillip K Allen/HOU/ECT@ECT, Steve Jackson/HOU/ECT@ECT
cc:  
Subject: 

We have room EB3014 from 3 - 4 pm on Wednesday.

WK



In [7]:

# drop the file column
df = df.drop('file', axis=1)

# got the domain for the emails
df['Sender_domain'] = df['Sender'].str.split('@').str[1]
df['Receiver_domain'] = df['Receiver'].str.split('@').str[1]


df.head()

,message,Sender,Receiver,n_receivers,CC,Subject,Body,Date,Sender_domain,Receiver_domain
0,Message-ID: <18782981.1075855378110.JavaMail.e...,phillip.allen@enron.com,tim.belden@enron.com,1,None,,Here is our forecast\n\n,2001-05-14 16:39:00-07:00,enron.com,enron.com
1,Message-ID: <15464986.1075855378456.JavaMail.e...,phillip.allen@enron.com,john.lavorato@enron.com,1,None,Re:,Traveling to have a business meeting takes the...,2001-05-04 13:51:00-07:00,enron.com,enron.com
2,Message-ID: <24216240.1075855687451.JavaMail.e...,phillip.allen@enron.com,leah.arsdall@enron.com,1,None,Re: test,test successful. way to go!!!,2000-10-18 03:00:00-07:00,enron.com,enron.com
3,Message-ID: <13505866.1075863688222.JavaMail.e...,phillip.allen@enron.com,randall.gay@enron.com,1,None,,"Randy,\n\n Can you send me a schedule of the s...",2000-10-23 06:13:00-07:00,enron.com,enron.com
4,Message-ID: <30922949.1075863688243.JavaMail.e...,phillip.allen@enron.com,greg.piper@enron.com,1,None,Re: Hello,Let's shoot for Tuesday at 11:45.,2000-08-31 05:07:00-07:00,enron.com,enron.com


# filter the dataset to decrease the dimension

In [8]:
# 1st filter: n_receiver=1, n_sender=1, no CC, use enron.com
 
filtered_df = df[(df["n_receivers"]== 1) & (df.CC.isna()) & (df.Sender_domain	== "enron.com") & (df.Receiver_domain	== "enron.com")]

# additional filtering based on value_counts
filtered_df = filtered_df[(filtered_df.Sender != "enron.announcements@enron.com") & (filtered_df.Receiver != "all.worldwide@enron.com") ]

print(df.shape, '\t', filtered_df.shape)

(517401, 10) 	 (162144, 10)


In [9]:
filtered_df.Receiver.value_counts()

# sender != enron.announcements@enron.com 
# receiver != all.worldwide@enron.com  

tana.jones@enron.com           3220
kate.symes@enron.com           2937
sara.shackleton@enron.com      2456
jeff.dasovich@enron.com        2115
steven.kean@enron.com          2064
                               ... 
'janet.dietrich'@enron.com        1
'jeff.donahue'@enron.com          1
guy.ishikawa@enron.com            1
jason.garvey@enron.com            1
clickathomecanada@enron.com       1
Name: Receiver, Length: 5010, dtype: int64

In [10]:
filtered_df['Date']

filtered_df['Date'] = pd.to_datetime(filtered_df['Date'], utc=True)

# Extract YYYY-MM-DD only
filtered_df['Date_short'] = filtered_df['Date'].dt.date


filter_date = pd.to_datetime('2001-05-01').date()

# Filter the DataFrame based on the 'Date_short' column
filtered_df = filtered_df[filtered_df['Date_short'] < filter_date]

filtered_df.head()

print(df.shape, filtered_df.shape)

(517401, 10) (110809, 11)


In [11]:
filtered_df.drop(["Sender_domain", 'Receiver_domain'], axis=1, inplace=True)

In [12]:
import re

def extract_original_message(text):
  parts = text.split("\n\n\n")
  original_parts = parts[0]
  return original_parts

filtered_df["Body_only"] = filtered_df["Body"].apply(extract_original_message)

In [13]:
print(filtered_df.iat[5000, filtered_df.columns.get_loc("Body_only")])

web_browser: Mozilla/4.0 (compatible; MSIE 5.01; Windows NT)

Greetings from NOVA Online.

A friend of yours has encrypted a message using NOVA's Virtual Enigma. The 
message, displayed below, is for your eyes only.

To decipher the message, you will need to access the Virtual Enigma (requires 
the Shockwave plugin), which is located on NOVA's Web site. You will also 
need a special 'key' to decode the message.

You will find the Virtual Enigma at the following url:
http://www.pbs.org/nova/decoding/virtwave.html

Message sent by:
eric (ebass@enron.com)

AAA

Encrypted message:

QTDHF AYTPE KHYYN WSJYE TBXNQ PJRRH GDVDX QHNNK QBKOD ZVQEU LTRMQ WDHUR ACFGZ 
VJDPV UFIZA M

NOVA Online
http://www.pbs.org/nova



In [14]:
filtered_df.iloc[5000,]

message        Message-ID: <2324497.1075854645870.JavaMail.ev...
Sender                                           ebass@enron.com
Receiver                                         ebass@enron.com
n_receivers                                                    1
CC                                                          None
Subject                                    NOVA's Virtual Enigma
Body           web_browser: Mozilla/4.0 (compatible; MSIE 5.0...
Date                                   2000-11-20 10:38:00+00:00
Date_short                                            2000-11-20
Body_only      web_browser: Mozilla/4.0 (compatible; MSIE 5.0...
Name: 15053, dtype: object

# merge the status and filtered_df

In [15]:
merged_df = pd.merge(status, filtered_df, left_on='Email',right_on="Sender", how='inner', suffixes=("Sender_", "sender_"))
merged_df = merged_df.rename(columns={'Name':'Sender_Name',
                                      "Email":"Sender_Email",
                                      "Position":"Sender_Position",
                                      "Rank":"Sender_Rank"})

In [16]:
merged_df = pd.merge(status, merged_df, left_on='Email',right_on="Receiver", how='inner')
merged_df = merged_df.rename(columns={'Name':'Receiver_Name',
                                      "Email":"Receiver_Email",
                                      "Position":"Receiver_Position",
                                      "Rank":"Receiver_Rank"})
merged_df.head()

,Receiver_Name,Receiver_Email,Receiver_Position,Receiver_Rank,Sender_Name,Sender_Email,Sender_Position,Sender_Rank,message,Sender,Receiver,n_receivers,CC,Subject,Body,Date,Date_short,Body_only
0,Andrew Lewis,andrew.lewis@enron.com,Vice President,4,Hunter Shively,hunter.shively@enron.com,Vice President,4,Message-ID: <20773283.1075848285687.JavaMail.e...,hunter.shively@enron.com,andrew.lewis@enron.com,1,None,Western Frontier,---------------------- Forwarded by Hunter S S...,2001-03-26 10:44:00+00:00,2001-03-26,---------------------- Forwarded by Hunter S S...
1,Andrew Lewis,andrew.lewis@enron.com,Vice President,4,Hunter Shively,hunter.shively@enron.com,Vice President,4,Message-ID: <5005866.1075848285868.JavaMail.ev...,hunter.shively@enron.com,andrew.lewis@enron.com,1,None,Western Frontier,---------------------- Forwarded by Hunter S S...,2001-03-26 10:44:00+00:00,2001-03-26,---------------------- Forwarded by Hunter S S...
2,Andrew Lewis,andrew.lewis@enron.com,Vice President,4,Hunter Shively,hunter.shively@enron.com,Vice President,4,Message-ID: <24323357.1075848286084.JavaMail.e...,hunter.shively@enron.com,andrew.lewis@enron.com,1,None,Western Frontier,---------------------- Forwarded by Hunter S S...,2001-03-26 10:44:00+00:00,2001-03-26,---------------------- Forwarded by Hunter S S...
3,Andrew Lewis,andrew.lewis@enron.com,Vice President,4,Hunter Shively,hunter.shively@enron.com,Vice President,4,Message-ID: <21388669.1075840995924.JavaMail.e...,hunter.shively@enron.com,andrew.lewis@enron.com,1,None,Western Frontier,\n---------------------- Forwarded by Hunter S...,2001-03-26 21:44:00+00:00,2001-03-26,\n---------------------- Forwarded by Hunter S...
4,Andrew Lewis,andrew.lewis@enron.com,Vice President,4,Hunter Shively,hunter.shively@enron.com,Vice President,4,Message-ID: <5246852.1075857472118.JavaMail.ev...,hunter.shively@enron.com,andrew.lewis@enron.com,1,None,European 10 jet stream heights,---------------------- Forwarded by Hunter S S...,2001-01-10 09:09:00+00:00,2001-01-10,---------------------- Forwarded by Hunter S S...


In [21]:
merged_df_short = merged_df[["Sender_Name","Sender_Email","Sender_Position","Sender_Rank", "Receiver_Name",	"Receiver_Email", "Receiver_Position",	"Receiver_Rank", "Body", "Date"]]

merged_df_short.head(2)

,Sender_Name,Sender_Email,Sender_Position,Sender_Rank,Receiver_Name,Receiver_Email,Receiver_Position,Receiver_Rank,Body,Date
0,Hunter Shively,hunter.shively@enron.com,Vice President,4,Andrew Lewis,andrew.lewis@enron.com,Vice President,4,---------------------- Forwarded by Hunter S S...,2001-03-26 10:44:00+00:00
1,Hunter Shively,hunter.shively@enron.com,Vice President,4,Andrew Lewis,andrew.lewis@enron.com,Vice President,4,---------------------- Forwarded by Hunter S S...,2001-03-26 10:44:00+00:00


In [23]:
merged_df_short_1 = merged_df_short[merged_df_short.Sender_Email != merged_df_short.Receiver_Email]

In [26]:
merged_df_short_1.tail(20)

,Sender_Name,Sender_Email,Sender_Position,Sender_Rank,Receiver_Name,Receiver_Email,Receiver_Position,Receiver_Rank,Body,Date
7009,Laura Luce,laura.luce@enron.com,Vice President,4,Vince Kaminski,vince.kaminski@enron.com,Manager,1,"Vince,\n\nYou beat me to the congrats. The su...",2000-01-11 10:50:00+00:00
7010,Laura Luce,laura.luce@enron.com,Vice President,4,Vince Kaminski,vince.kaminski@enron.com,Manager,1,"Vince,\n\nYou beat me to the congrats. The su...",2000-01-11 10:50:00+00:00
7011,Louise Kitchen,louise.kitchen@enron.com,President,5,Vince Kaminski,vince.kaminski@enron.com,Manager,1,Absolutely - I can't find the previous email b...,2000-02-11 17:13:00+00:00
7012,Louise Kitchen,louise.kitchen@enron.com,President,5,Vince Kaminski,vince.kaminski@enron.com,Manager,1,Congratulations to you too.\n\nHappy New Year ...,2000-01-11 16:17:00+00:00
7013,Louise Kitchen,louise.kitchen@enron.com,President,5,Vince Kaminski,vince.kaminski@enron.com,Manager,1,Thanks - we are going to set up interviews. ...,2000-12-05 18:21:00+00:00
7014,Louise Kitchen,louise.kitchen@enron.com,President,5,Vince Kaminski,vince.kaminski@enron.com,Manager,1,Just thought you'd like to know we are not hav...,2000-03-27 13:43:00+00:00
7015,Louise Kitchen,louise.kitchen@enron.com,President,5,Vince Kaminski,vince.kaminski@enron.com,Manager,1,Thanks - we are going to set up interviews. ...,2000-12-05 18:21:00+00:00
7016,Louise Kitchen,louise.kitchen@enron.com,President,5,Vince Kaminski,vince.kaminski@enron.com,Manager,1,Congratulations to you too.\n\nHappy New Year ...,2000-01-11 16:17:00+00:00
7017,Louise Kitchen,louise.kitchen@enron.com,President,5,Vince Kaminski,vince.kaminski@enron.com,Manager,1,Absolutely - I can't find the previous email b...,2000-02-11 17:13:00+00:00
7018,Louise Kitchen,louise.kitchen@enron.com,President,5,Vince Kaminski,vince.kaminski@enron.com,Manager,1,Just thought you'd like to know we are not hav...,2000-03-27 13:43:00+00:00


In [28]:
merged_df_short_1['content'] = merged_df_short['Body'].str.split("---------------------- Forwarded by").str[0] 

import re

merged_df_short_1['content']= merged_df_short_1["content"].replace(r"Monthly Basis Swap", "",  regex=True)
merged_df_short_1['content'] = merged_df_short_1["content"].replace(r"Daily Basis Swap", "",  regex=True)
merged_df_short_1['content'] = merged_df_short_1["content"].replace(r"Daily for Monthly Index Swap", "",  regex=True)
merged_df_short_1['content'] = merged_df_short_1["content"].replace(r"Fixed Price Physical", "",   regex=True)
merged_df_short_1['content'] = merged_df_short_1["content"].replace(r"Physical Index", "",   regex=True)
merged_df_short_1['content'] = merged_df_short_1["content"].replace(r"Fixed Price Swap", "", regex=True)

In [29]:
merged_df_short['content'] = merged_df_short['content'].str.split("---------------------- Forwarded by").str[0] 

In [31]:
merged_df_short_1.shape

(6531, 11)

In [32]:
merged_df_short_1["Sender-Receiver"] = merged_df_short_1["Sender_Rank"] - merged_df_short_1["Receiver_Rank"]
merged_df_short_1.head()

,Sender_Name,Sender_Email,Sender_Position,Sender_Rank,Receiver_Name,Receiver_Email,Receiver_Position,Receiver_Rank,Body,Date,content,Sender-Receiver
0,Hunter Shively,hunter.shively@enron.com,Vice President,4,Andrew Lewis,andrew.lewis@enron.com,Vice President,4,---------------------- Forwarded by Hunter S S...,2001-03-26 10:44:00+00:00,,0
1,Hunter Shively,hunter.shively@enron.com,Vice President,4,Andrew Lewis,andrew.lewis@enron.com,Vice President,4,---------------------- Forwarded by Hunter S S...,2001-03-26 10:44:00+00:00,,0
2,Hunter Shively,hunter.shively@enron.com,Vice President,4,Andrew Lewis,andrew.lewis@enron.com,Vice President,4,---------------------- Forwarded by Hunter S S...,2001-03-26 10:44:00+00:00,,0
3,Hunter Shively,hunter.shively@enron.com,Vice President,4,Andrew Lewis,andrew.lewis@enron.com,Vice President,4,\n---------------------- Forwarded by Hunter S...,2001-03-26 21:44:00+00:00,\n,0
4,Hunter Shively,hunter.shively@enron.com,Vice President,4,Andrew Lewis,andrew.lewis@enron.com,Vice President,4,---------------------- Forwarded by Hunter S S...,2001-01-10 09:09:00+00:00,,0


In [33]:
def get_direction(x):
    if x == 0:
        return "same"
    elif x < 0:
        return "upward"
    else:
        return "downward"
    
merged_df_short_1["direction"] = merged_df_short_1["Sender-Receiver"].apply(get_direction)



In [35]:
merged_df_short_1.direction.value_counts()

same        2299
downward    2234
upward      1998
Name: direction, dtype: int64

# NLTK library

In [38]:
# tokenization

import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

nltk.download('punkt')

# initialize the Potter stemmer
porter = PorterStemmer()

# define a function to tokenize and stem a text string
def tokenize_and_stem(text):
    tokens = word_tokenize(text.lower())
    stems = [porter.stem(token) for token in tokens]
    return stems

# apply the tokenize_and_stem function to the content column
merged_df_short_1['tokens'] = merged_df_short_1['content'].apply(tokenize_and_stem)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [53]:
print(merged_df_short_1.Receiver_Position.value_counts())
print("\n")
print(merged_df_short_1.Sender_Position.value_counts())

# Rank 0 = Employee, In House Lawyer, Trader

Vice President         1854
Employee               1231
Managing Director       973
CEO                     633
President               563
Director                489
Manager                 331
Trader                  245
In House Lawyer         202
Director of Trading      10
Name: Receiver_Position, dtype: int64


Vice President         2035
Employee               1443
CEO                    1094
Managing Director       611
Manager                 297
Trader                  292
President               291
In House Lawyer         280
Director                187
Director of Trading       1
Name: Sender_Position, dtype: int64


In [44]:
# group the DataFrame by "direction" and apply the mean to "tokens" column
grouped_df = merged_df_short_1.groupby("direction")["tokens"].apply(lambda x: x.apply(len).mean())
grouped_df

direction
downward    123.386750
same        103.377555
upward      113.666166
Name: tokens, dtype: float64

In [50]:
merged_df_short_1.groupby("Sender-Receiver")["tokens"].apply(lambda x: x.apply(len).mean())

Sender-Receiver
-6     48.700000
-5    108.956204
-4    101.383632
-3    112.915584
-2     65.287383
-1    151.006906
 0    103.377555
 1    279.579173
 2     31.312317
 3     76.261468
 4     70.906015
 5     46.508929
 6     26.648649
Name: tokens, dtype: float64

In [47]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string
nltk.download('stopwords')

# initialize the stop words and punctuation
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)

# define a function to clean a single token
def clean_token(token):
    # remove punctuation and numbers
    token = ''.join([char for char in token if char not in punctuation and not char.isdigit()])
    # convert to lowercase
    token = token.lower()
    # remove stopwords
    if token not in stop_words:
        return token

# define a function to clean a list of tokens
def clean_tokens(tokens):
    cleaned_tokens = []
    for token in tokens:
        cleaned_token = clean_token(token)
        if cleaned_token:
            cleaned_tokens.append(cleaned_token)
    return cleaned_tokens

# apply the clean_tokens function to the "tokens" column and store the cleaned tokens in a new "cleaned_tokens" column
merged_df_short_1['cleaned_tokens'] = merged_df_short_1['tokens'].apply(clean_tokens)
merged_df_short_1.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Sender_Name,Sender_Email,Sender_Position,Sender_Rank,Receiver_Name,Receiver_Email,Receiver_Position,Receiver_Rank,Body,Date,content,Sender-Receiver,direction,tokens,cleaned_tokens
0,Hunter Shively,hunter.shively@enron.com,Vice President,4,Andrew Lewis,andrew.lewis@enron.com,Vice President,4,---------------------- Forwarded by Hunter S S...,2001-03-26 10:44:00+00:00,,0,same,[],[]
1,Hunter Shively,hunter.shively@enron.com,Vice President,4,Andrew Lewis,andrew.lewis@enron.com,Vice President,4,---------------------- Forwarded by Hunter S S...,2001-03-26 10:44:00+00:00,,0,same,[],[]
2,Hunter Shively,hunter.shively@enron.com,Vice President,4,Andrew Lewis,andrew.lewis@enron.com,Vice President,4,---------------------- Forwarded by Hunter S S...,2001-03-26 10:44:00+00:00,,0,same,[],[]
3,Hunter Shively,hunter.shively@enron.com,Vice President,4,Andrew Lewis,andrew.lewis@enron.com,Vice President,4,\n---------------------- Forwarded by Hunter S...,2001-03-26 21:44:00+00:00,\n,0,same,[],[]
4,Hunter Shively,hunter.shively@enron.com,Vice President,4,Andrew Lewis,andrew.lewis@enron.com,Vice President,4,---------------------- Forwarded by Hunter S S...,2001-01-10 09:09:00+00:00,,0,same,[],[]


In [48]:
# group the DataFrame by "direction" and apply the mean to "tokens" column
grouped_df_cl = merged_df_short_1.groupby("direction")["cleaned_tokens"].apply(lambda x: x.apply(len).mean())
grouped_df_cl

direction
downward    61.745300
same        49.240104
upward      56.749750
Name: cleaned_tokens, dtype: float64

In [49]:
merged_df_short_1.groupby("Sender-Receiver")["cleaned_tokens"].apply(lambda x: x.apply(len).mean())

Sender-Receiver
-6     22.400000
-5     57.270073
-4     52.332481
-3     55.103896
-2     31.443925
-1     75.171271
 0     49.240104
 1    141.704708
 2     14.674487
 3     38.206422
 4     33.699248
 5     22.580357
 6     12.027027
Name: cleaned_tokens, dtype: float64

In [46]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string
nltk.download('stopwords')

# initialize the stop words and punctuation
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)

# define a function to clean a single token
def clean_token(token):
    # remove punctuation and numbers
    token = ''.join([char for char in token if char not in punctuation and not char.isdigit()])
    # convert to lowercase
    token = token.lower()
    # remove stopwords
    if token not in stop_words:
        return token

# define a function to clean a list of tokens
def clean_tokens(tokens):
    cleaned_tokens = []
    for token in tokens:
        cleaned_token = clean_token(token)
        if cleaned_token:
            cleaned_tokens.append(cleaned_token)
    return cleaned_tokens

# create a sample DataFrame with a "tokens" column
df = pd.DataFrame({'tokens': [['This', 'is', 'a', 'sample', 'sentence', '123', '.'], ['Another', 'sentence', 'here', '456', '!']]})

# apply the clean_tokens function to the "tokens" column and store the cleaned tokens in a new "cleaned_tokens" column
df['cleaned_tokens'] = df['tokens'].apply(clean_tokens)

# print the resulting DataFrame
print(df)


                                    tokens       cleaned_tokens
0  [This, is, a, sample, sentence, 123, .]   [sample, sentence]
1        [Another, sentence, here, 456, !]  [another, sentence]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [36]:
merged_df_short_1.content.value_counts()

In [60]:
merged_df_short_1_only_employee = merged_df_short_1[~merged_df_short_1.Receiver_Position.str.contains("In House Lawyer|Trader|Director of Trading")]
merged_df_short_1_only_employee = merged_df_short_1_only_employee[~merged_df_short_1_only_employee.Sender_Position.str.contains("In House Lawyer|Trader|Director of Trading")]

merged_df_short_1_only_employee.Receiver_Position.value_counts()

merged_df_short_1_only_employee.head()

,Sender_Name,Sender_Email,Sender_Position,Sender_Rank,Receiver_Name,Receiver_Email,Receiver_Position,Receiver_Rank,Body,Date,content,Sender-Receiver,direction,tokens,cleaned_tokens
0,Hunter Shively,hunter.shively@enron.com,Vice President,4,Andrew Lewis,andrew.lewis@enron.com,Vice President,4,---------------------- Forwarded by Hunter S S...,2001-03-26 10:44:00+00:00,,0,same,[],[]
1,Hunter Shively,hunter.shively@enron.com,Vice President,4,Andrew Lewis,andrew.lewis@enron.com,Vice President,4,---------------------- Forwarded by Hunter S S...,2001-03-26 10:44:00+00:00,,0,same,[],[]
2,Hunter Shively,hunter.shively@enron.com,Vice President,4,Andrew Lewis,andrew.lewis@enron.com,Vice President,4,---------------------- Forwarded by Hunter S S...,2001-03-26 10:44:00+00:00,,0,same,[],[]
3,Hunter Shively,hunter.shively@enron.com,Vice President,4,Andrew Lewis,andrew.lewis@enron.com,Vice President,4,\n---------------------- Forwarded by Hunter S...,2001-03-26 21:44:00+00:00,\n,0,same,[],[]
4,Hunter Shively,hunter.shively@enron.com,Vice President,4,Andrew Lewis,andrew.lewis@enron.com,Vice President,4,---------------------- Forwarded by Hunter S S...,2001-01-10 09:09:00+00:00,,0,same,[],[]


In [64]:
# group the DataFrame by "direction" and apply the mean to "tokens" column
merged_df_short_1_only_employee.groupby("direction")["cleaned_tokens"].apply(lambda x: x.apply(len).mean())
merged_df_short_1_only_employee.groupby("direction")["tokens"].apply(lambda x: x.apply(len).mean())


merged_df_short_1_only_employee.groupby("Sender-Receiver")["tokens"].apply(lambda x: x.apply(len).mean())

merged_df_short_1_only_employee.groupby("Sender-Receiver")["cleaned_tokens"].apply(lambda x: x.apply(len).mean())

Sender-Receiver
-6      8.000000
-5     57.029412
-4     44.904762
-3     56.900000
-2     33.073232
-1     78.225619
 0     49.526055
 1    147.142003
 2     14.594955
 3     39.373786
 4     27.808889
 5     22.580357
 6     13.777778
Name: cleaned_tokens, dtype: float64

# etc


In [ ]:
merged_df_short = merged_df[["Sender_Name","Sender_Email","Sender_Position","Sender_Rank", "Receiver_Name",	"Receiver_Email", "Receiver_Position",	"Receiver_Rank", "Body"]]
print(merged_df_short.iloc[1, -1])

In [ ]:
x = merged_df_short.iloc[1, -1]
x.split("---------------------- Forwarded by")[0]

In [ ]:
merged_df_short['content'] = merged_df_short['Body'].str.split("---------------------- Forwarded by").str[0] 

merged_df_short

In [ ]:
merged_df_short_1.info()

In [ ]:
merged_df_short_1['content'] = merged_df_short_1['content'].str.split("\n\n\n").str[0] 

In [ ]:
import re

merged_df_short_1['content']= merged_df_short_1["content"].replace(r"Monthly Basis Swap", "",  regex=True)
merged_df_short_1['content'] = merged_df_short_1["content"].replace(r"Daily Basis Swap", "",  regex=True)
merged_df_short_1['content'] = merged_df_short_1["content"].replace(r"Daily for Monthly Index Swap", "",  regex=True)
merged_df_short_1['content'] = merged_df_short_1["content"].replace(r"Fixed Price Physical", "",   regex=True)
merged_df_short_1['content'] = merged_df_short_1["content"].replace(r"Physical Index", "",   regex=True)
merged_df_short_1['content'] = merged_df_short_1["content"].replace(r"Fixed Price Swap", "", regex=True)

In [ ]:
merged_df_short_1.iloc[114, -1]

In [ ]:
print(merged_df_short_1.iloc[120, -1])

In [ ]:
print(merged_df_short_1.content.sample(1))